# Étape 1 — Définir les constantes « overview » (traduction FR)

**Objectif.** Centraliser les paramètres globaux de l’étude (croissance, main-d’œuvre, actualisation, horizon) dans un fichier unique `overview.py`.
**Remarques clés.**

* Les **taux** sont en fraction (ex. `0.02` = 2 %).
* Les **heures supplémentaires** se paient à **1,5×** le salaire de base. Comme le « 1,0× » est déjà compté dans les heures normales, on ajoute seulement la **majoration** de `0,5×` pour les heures sup (d’où `facteur_majoration_heures_sup = 0.5`).
* L’énoncé demande de comparer les options sur un **horizon de 5 ans** et d’évaluer la **VAN** pour choisir entre agrandir le stade existant ou en construire un nouveau. 


In [75]:
import pandas as pd

# 1. Forcer pandas à ne jamais afficher en notation scientifique
pd.options.display.float_format = '{:,.2f}'.format

# 2. Ajouter séparateurs d'espaces pour les grands nombres (style français)
pd.options.display.float_format = lambda x: f"{x:,.2f}".replace(",", " ")


## Étape 2 — Données Main-d'œuvre

Nous importons la feuille *Main-d'œuvre* du fichier Excel :  
- On saute les 3 premières lignes d’instructions.  
- On conserve **toutes les colonnes** (toutes sont utiles pour l’analyse).  
- On ignore la première colonne (inutile).  
- On ajoute une colonne calculée : **Salaire annuel (année 1)** en tenant compte :
  - des heures normales (heures moyennes × salaire horaire × 12),
  - des heures supplémentaires : une heure supplémentaire est payée **1,5 × le salaire horaire**.  
    Comme les **1,0 ×** sont déjà comptés dans les heures normales, on ajoute seulement **+0,5 × salaire horaire × heures sup × 12**.


In [ ]:
import pandas as pd
import sys, os
sys.path.insert(0, os.path.abspath(".."))

from utils.constants import VUEDENSEMBLE

# Charger le fichier Excel (adapter le chemin si besoin)
fichier_excel = "../data/Toronto Sports Entertainment_Sept 2025_FR.xlsx"

# Lire la feuille "Main-d'oeuvre" en sautant les 3 premières lignes
df_main = pd.read_excel(fichier_excel, sheet_name="Informations - main d'oeuvre", skiprows=3)
df_main = df_main.iloc[:, 1:]  # Supprimer la première colonne vide
df_main["Option de projet d’infrastructure"] = df_main["Option de projet d’infrastructure"].replace({
    "Agrandir le TO Field": "Agrandissement du TO Field",
    "Construire un nouveau stade": "Construction d’un nouveau stade"
})

# Calcul du salaire annuel de la première année
heures_base_mois = VUEDENSEMBLE["heures_moyennes_par_mois"]
facteur_sup = VUEDENSEMBLE["facteur_majoration_heures_sup"]

df_main["Salaire annuel (année 1)"] = (
    (heures_base_mois * df_main["Salaire horaire ($)"] * 12)
    + (df_main["Heures supplémentaires/mois"] * df_main["Salaire horaire ($)"] * (1 + facteur_sup) * 12)
)

print("Constantes utilisées :")
print(f"Heures moyennes par mois = {heures_base_mois}")
print(f"Facteur de majoration heures supplémentaires = {facteur_sup}")

df_main.head()


Constantes utilisées :
Heures moyennes par mois = 130
Facteur de majoration heures supplémentaires = 0.5


,Option de projet d’infrastructure,Rôle,Niveau d'éducation,Salaire horaire ($),Heures supplémentaires/mois,Salaire annuel (année 1)
0,Construire un nouveau stade,Ingénieur,Maîtrise,40,1,63 120.00
1,Agrandir le TO Field,Gestionnaire de projet,Doctorat,84,6,140 112.00
2,Construire un nouveau stade,Travailleur de la construction,Maîtrise,63,2,100 548.00
3,Agrandir le TO Field,Électricien,Maîtrise,81,8,138 024.00
4,Agrandir le TO Field,Travailleur de la construction,École secondaire,46,39,104 052.00


## Étape 3 — Projection des salaires sur 5 ans

À partir du **salaire annuel (année 1)** calculé à l’étape 2 :  
- Nous appliquons le **taux de croissance des coûts** défini dans `constants.py`.  
- Pour chaque rôle, nous générons 4 nouvelles colonnes : `Salaire annuel (année 2)`, ..., `Salaire annuel (année 5)`.  
- Cela permet d’obtenir le **coût de main-d’œuvre projeté sur 5 ans**.


In [77]:
# Récupération du taux de croissance des coûts
taux_croissance = VUEDENSEMBLE["taux_croissance_couts"]

# Création des colonnes pour les années 2 à 5
for annee in range(2, VUEDENSEMBLE["duree_analyse_ans"] + 1):
    col_prec = f"Salaire annuel (année {annee-1})"
    col_nouv = f"Salaire annuel (année {annee})"
    df_main[col_nouv] = df_main[col_prec] * (1 + taux_croissance)

print(f"Taux de croissance des coûts = {taux_croissance}")

# Aperçu des 5 premières lignes
df_main.head()


Taux de croissance des coûts = 0.02


,Option de projet d’infrastructure,Rôle,Niveau d'éducation,Salaire horaire ($),Heures supplémentaires/mois,Salaire annuel (année 1),Salaire annuel (année 2),Salaire annuel (année 3),Salaire annuel (année 4),Salaire annuel (année 5)
0,Construire un nouveau stade,Ingénieur,Maîtrise,40,1,63 120.00,64 382.40,65 670.05,66 983.45,68 323.12
1,Agrandir le TO Field,Gestionnaire de projet,Doctorat,84,6,140 112.00,142 914.24,145 772.52,148 687.98,151 661.73
2,Construire un nouveau stade,Travailleur de la construction,Maîtrise,63,2,100 548.00,102 558.96,104 610.14,106 702.34,108 836.39
3,Agrandir le TO Field,Électricien,Maîtrise,81,8,138 024.00,140 784.48,143 600.17,146 472.17,149 401.62
4,Agrandir le TO Field,Travailleur de la construction,École secondaire,46,39,104 052.00,106 133.04,108 255.70,110 420.81,112 629.23


## Étape 4 — Agrégation des salaires par projet

Nous regroupons les données par **option de projet d’infrastructure** :  
- Pour chaque projet (Construire un nouveau stade, Agrandir le TO Field),  
  - nous additionnons les salaires par année (année 1 à 5),  
  - nous comptons le nombre total de travailleurs associés.  

Cela permet de comparer directement l’évolution des coûts de main-d’œuvre pour les deux projets sur 5 ans.


In [78]:
# Agréger par option de projet d'infrastructure
df_salaire_projet = df_main.groupby("Option de projet d’infrastructure").agg(
    {
        "Salaire annuel (année 1)": "sum",
        "Salaire annuel (année 2)": "sum",
        "Salaire annuel (année 3)": "sum",
        "Salaire annuel (année 4)": "sum",
        "Salaire annuel (année 5)": "sum",
        "Rôle": "count"  # nombre de travailleurs
    }
)

# Renommer la colonne "Rôle" pour plus de clarté
df_salaire_projet = df_salaire_projet.rename(columns={"Rôle": "Nombre de travailleurs"})

df_salaire_projet


,Salaire annuel (année 1),Salaire annuel (année 2),Salaire annuel (année 3),Salaire annuel (année 4),Salaire annuel (année 5),Nombre de travailleurs
Option de projet d’infrastructure,,,,,,
Agrandir le TO Field,12 804 216.00,13 060 300.32,13 321 506.33,13 587 936.45,13 859 695.18,108
Construire un nouveau stade,10 848 822.00,11 065 798.44,11 287 114.41,11 512 856.70,11 743 113.83,92


## Étape 5 — Coûts de construction

Nous importons la feuille *Coûts de construction* du fichier Excel :  
- On saute les 3 premières lignes d’instructions.  
- On conserve toutes les colonnes.  
- Pour chaque option de projet (Construire un nouveau stade, Agrandir le TO Field),  
  - nous identifions les **coûts de construction** initiaux,  
  - nous préparons une projection sur 5 ans (si applicable, en appliquant le taux de croissance des coûts).  

Cette étape est cruciale, car les coûts de construction représentent une **dépense majeure initiale** qui impacte directement la VAN.  
Ces chiffres, combinés avec les coûts de main-d’œuvre et les revenus futurs, permettront d’évaluer la rentabilité relative des deux options.


In [114]:
# Lecture de la feuille "Construction" en sautant les 3 premières lignes
df_construction = pd.read_excel(fichier_excel, sheet_name="Coûts de construction", skiprows=3)
df_construction = df_construction.iloc[:, 1:]  # Supprimer la première colonne vide

# Conversion en float pour forcer l'affichage avec séparateurs d'espaces
df_construction = df_construction.astype(float, errors="ignore")

# Aperçu pour vérifier les colonnes
print("Aperçu des coûts de construction :")
display(df_construction.head())

Aperçu des coûts de construction :


,Option de projet d’infrastructure,Coût d’achat du terrain,Coûts de permis et d’inspection du site,Coûts de préparation du site,Coûts annuels d’entretien,Coûts annuels de location d’équipement,Coûts mensuels de fournitures et matériaux,Coûts mensuels des services publics,Part mensuelle des revenus partagés avec les sponsors
0,Agrandissement du TO Field,25 000 000.00,29 000.00,131 000.00,500 000.00,76 000.00,55 000.00,32 000.00,20 000.00
1,Construction d’un nouveau stade,343 000 000.00,69 000.00,428 000.00,630 000.00,103 000.00,119 000.00,32 000.00,10 000.00


## Étape 5.1 — Analyse et insights des coûts de construction

À partir du tableau projeté (Année 0 → Année 5), nous ajoutons deux enrichissements :  

1. **Total (0-5 ans)** : somme des coûts sur toute la période d’analyse.  
2. **% du coût initial sur le total** : mesure du poids des coûts upfront dans le projet.  

### Conclusions possibles
- Un pourcentage élevé signifie un projet très **capitalistique** avec un risque initial important.  
- Un pourcentage plus faible signifie que le projet est **plus étalé dans le temps**, mais avec des charges récurrentes plus importantes.  
Ces différences structurent la manière dont les projets impactent la trésorerie et la prise de risque, et constituent des arguments dans le choix final.


In [107]:
taux_croissance = VUEDENSEMBLE["taux_croissance_couts"]

# Colonnes
col_uniques = ["Coût d’achat du terrain", "Coûts de permis et d’inspection du site", "Coûts de préparation du site"]
col_annuels = ["Coûts annuels d’entretien", "Coûts annuels de location d’équipement"]
col_mensuels = ["Coûts mensuels de fournitures et matériaux", "Coûts mensuels des services publics", "Part mensuelle des revenus partagés avec les sponsors"]

# DataFrame de projection avec même index et colonnes claires
df_construction_proj = pd.DataFrame(index=df_construction["Option de projet d’infrastructure"].values)

# Année 0 : seulement les coûts uniques
df_construction_proj["Année 0"] = df_construction[col_uniques].sum(axis=1).values

# Années 1 à 5
for annee in range(1, VUEDENSEMBLE["duree_analyse_ans"] + 1):
    facteur = (1 + taux_croissance) ** (annee - 1)
    couts_annuels = df_construction[col_annuels].sum(axis=1).values * facteur
    couts_mensuels = df_construction[col_mensuels].sum(axis=1).values * 12 * facteur
    df_construction_proj[f"Année {annee}"] = couts_annuels + couts_mensuels

df_construction_proj["Total (0-5 ans)"] = df_construction_proj.sum(axis=1)
df_construction_proj["% coût initial sur total"] = (
    df_construction_proj["Année 0"] / df_construction_proj["Total (0-5 ans)"] * 100
)
df_construction_proj = df_construction_proj.astype(float)
df_construction_proj


,Année 0,Année 1,Année 2,Année 3,Année 4,Année 5,Total (0-5 ans),% coût initial sur total
Agrandissement du TO Field,25 160 000.00,1 860 000.00,1 897 200.00,1 935 144.00,1 973 846.88,2 013 323.82,34 839 514.70,72.22
Construction d’un nouveau stade,343 497 000.00,2 665 000.00,2 718 300.00,2 772 666.00,2 828 119.32,2 884 681.71,357 365 767.03,96.12


## Étape 6 — Lecture des revenus

Nous importons la feuille *Revenus* du fichier Excel :  
- On saute les 3 premières lignes d’instructions.  
- On supprime la première colonne inutile (colonne vide).  
- On affiche un aperçu des données brutes pour vérifier la structure et les colonnes disponibles.  

Cette étape sert uniquement à **explorer et valider la table des revenus**, avant de passer à l’étape 6.1 où nous ferons les projections et les analyses.


In [113]:
# Lecture de la feuille "Revenus" en sautant les 3 premières lignes
df_revenus = pd.read_excel(fichier_excel, sheet_name="Revenus", skiprows=3)
df_revenus = df_revenus.iloc[:, 1:]  # Supprimer la première colonne vide

# Conversion en float pour forcer l'affichage avec séparateurs d'espaces
df_revenus = df_revenus.astype(float, errors="ignore")

# Aperçu des revenus
print("Aperçu des revenus :")
display(df_revenus.head())


Aperçu des revenus :


,Option de projet d’infrastructure,Revenus annuels de la billetterie ($),Revenus annuels de sponsoring ($),Revenus annuels des concessions ($),Ventes annuelles de marchandises ($)
0,Agrandissement du TO Field,1 335 800.00,2 954 470.00,420 660.00,166 110.00
1,Construction d’un nouveau stade,2 070 110.00,2 822 390.00,321 600.00,798 950.00


## Étape 6.1 — Projection et analyse des revenus

À partir des revenus de base (année 1), nous projetons les revenus sur 5 ans :  
- Une croissance annuelle de **15 %** est appliquée.  
- On calcule le **Total cumulé (5 ans)**.  
- On décompose les revenus par catégorie pour analyser leur poids relatif (%).  

### Conclusions possibles
- Identifier quelle source de revenus est dominante (ex. sponsoring vs billetterie).  
- Comprendre si un projet est plus dépendant d’une source de revenus spécifique (risque de concentration).  
- Comparer la dynamique de croissance entre les deux projets.


In [121]:
from IPython.display import display

# Colonnes des catégories de revenus
colonnes_revenus = [
    "Revenus annuels de la billetterie ($)",
    "Revenus annuels de sponsoring ($)",
    "Revenus annuels des concessions ($)",
    "Ventes annuelles de marchandises ($)"
]

# Dictionnaire pour stocker les tableaux séparés
projets_revenus = {}

for idx, row in df_revenus.iterrows():
    projet = row["Option de projet d’infrastructure"]
    data = {}

    # Projection année par année
    for col in colonnes_revenus:
        data[col] = [
            row[col] * (1 + taux_revenus) ** (annee - 1)
            for annee in range(1, VUEDENSEMBLE["duree_analyse_ans"] + 1)
        ]
    
    # Créer un DataFrame avec Année 1 → 5
    df_proj = pd.DataFrame(data, index=[f"Année {i}" for i in range(1, 6)])
    
    # Ajouter total cumulé par catégorie
    df_proj.loc["Total (5 ans)"] = df_proj.sum()
    
    # Ajouter % catégorie (Année 1)
    part_totale = df_proj.loc["Année 1"].sum()
    for col in colonnes_revenus:
        df_proj[f"% {col} (Année 1)"] = (df_proj.loc["Année 1", col] / part_totale) * 100
    
    projets_revenus[projet] = df_proj

print("Agrandissement du TO Field")
display(projets_revenus["Agrandissement du TO Field"])

print("Construction d’un nouveau stade")
display(projets_revenus["Construction d’un nouveau stade"])



Agrandissement du TO Field


,Revenus annuels de la billetterie ($),Revenus annuels de sponsoring ($),Revenus annuels des concessions ($),Ventes annuelles de marchandises ($),% Revenus annuels de la billetterie ($) (Année 1),% Revenus annuels de sponsoring ($) (Année 1),% Revenus annuels des concessions ($) (Année 1),% Ventes annuelles de marchandises ($) (Année 1)
Année 1,1 335 800.00,2 954 470.00,420 660.00,166 110.00,27.39,60.58,8.63,3.41
Année 2,1 536 170.00,3 397 640.50,483 759.00,191 026.50,27.39,60.58,8.63,3.41
Année 3,1 766 595.50,3 907 286.57,556 322.85,219 680.47,27.39,60.58,8.63,3.41
Année 4,2 031 584.82,4 493 379.56,639 771.28,252 632.55,27.39,60.58,8.63,3.41
Année 5,2 336 322.55,5 167 386.50,735 736.97,290 527.43,27.39,60.58,8.63,3.41
Total (5 ans),9 006 472.87,19 920 163.13,2 836 250.10,1 119 976.95,27.39,60.58,8.63,3.41


Construction d’un nouveau stade


,Revenus annuels de la billetterie ($),Revenus annuels de sponsoring ($),Revenus annuels des concessions ($),Ventes annuelles de marchandises ($),% Revenus annuels de la billetterie ($) (Année 1),% Revenus annuels de sponsoring ($) (Année 1),% Revenus annuels des concessions ($) (Année 1),% Ventes annuelles de marchandises ($) (Année 1)
Année 1,2 070 110.00,2 822 390.00,321 600.00,798 950.00,34.43,46.94,5.35,13.29
Année 2,2 380 626.50,3 245 748.50,369 840.00,918 792.50,34.43,46.94,5.35,13.29
Année 3,2 737 720.47,3 732 610.77,425 316.00,1 056 611.37,34.43,46.94,5.35,13.29
Année 4,3 148 378.55,4 292 502.39,489 113.40,1 215 103.08,34.43,46.94,5.35,13.29
Année 5,3 620 635.33,4 936 377.75,562 480.41,1 397 368.54,34.43,46.94,5.35,13.29
Total (5 ans),13 957 470.85,19 029 629.42,2 168 349.81,5 386 825.50,34.43,46.94,5.35,13.29


## Étape 6.2 — Tableau comparatif des revenus totaux

Nous construisons un tableau résumé par projet :  
- **Total cumulé (5 ans)** de tous les revenus  
- **Moyenne annuelle**  
- **Revenu par travailleur (5 ans)** (en lien avec l’étape 4)

### Intérêt
- Ce tableau offre une vision synthétique et directement comparable des deux projets.  
- Il permet d’argumenter plus facilement : ex. "le nouveau stade génère X % de plus de revenus totaux", ou "il est 20 % plus productif par travailleur".


In [124]:
# Recalculer le total cumulé 5 ans pour chaque projet
revenus_totaux = {}

for projet, df_proj in projets_revenus.items():
    total_revenus = df_proj.loc["Total (5 ans)", colonnes_revenus].sum()
    revenus_totaux[projet] = {
        "Total (5 ans)": total_revenus,
        "Moyenne annuelle": total_revenus / VUEDENSEMBLE["duree_analyse_ans"],
    }

# Transformer en DataFrame
df_revenus_total = pd.DataFrame(revenus_totaux).T

print(df_revenus_total.index)
print(df_salaire_projet.index)

# Ajouter revenu par travailleur (en utilisant df_salaire_projet de l’étape 4)
df_revenus_total["Revenu par travailleur (5 ans)"] = (
    df_revenus_total["Total (5 ans)"] / df_salaire_projet["Nombre de travailleurs"]
)

df_revenus_total


Index(['Agrandissement du TO Field', 'Construction d’un nouveau stade'], dtype='object')
Index(['Agrandir le TO Field', 'Construire un nouveau stade'], dtype='object', name='Option de projet d’infrastructure')


,Total (5 ans),Moyenne annuelle,Revenu par travailleur (5 ans)
Agrandissement du TO Field,32 882 863.05,6 576 572.61,NaN
Construction d’un nouveau stade,40 542 275.58,8 108 455.12,NaN
